In [1]:
using Ai4EComponentLib
using Ai4EComponentLib.CompressedAirSystem
using ModelingToolkit, DifferentialEquations
using Plots

In [2]:
n0 = 4000
h_polCoff = [-91.7802, 1058.2670, 3213.1520]
etaCoff = [-0.0181, 0.2880, -0.2557]
surgeCoff = [-2.950e-7, 4.8009, -5.1678]
chokeCoff = [1.1054e-6, 8.6274, 20.7626]

f=0.05                  # Friction resistance coefficient
n = 15                  # Number of nodes
D = 0.4                 # Pipe diameter
L = 20                  # length
R = 3000                # Resistance coefficient
T = 300                 # Temperature
qm0 = 10 * ones(n)      # Initial mass flow rate
p0 = 10e5 * ones(n)     # initial pressure

n2 = 40                 # Number of nodes
D2 = 0.4                # Pipe diameter
L2 = 200                # length
R2 = 3000               # Resistance coefficient
T2 = 300                # Temperature
qm02 = 10 * ones(n2)    # Initial mass flow rate
p02 = range(10e5,8e5,length=n2)  #initial pressure

inletBoundary = Dict(
    "p" => 1.0e5,
    "T" => 300,
)

outletBoundary = Dict(
    "T" => 300,
    "p" => 4e5,
)

Dict{String, Real} with 2 entries:
  "T" => 300
  "p" => 400000.0

In [3]:
@named inletSource = Source(boundary=inletBoundary)
@named outletSource = Source(boundary=outletBoundary)

┌ Warning: source contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit /Users/jerell/.julia/packages/ModelingToolkit/hBVcX/src/systems/connectors.jl:40
┌ Warning: source contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit /Users/jerell/.julia/packages/ModelingToolkit/hBVcX/src/systems/connectors.jl:40


Model outletSource with 5 (6) equations
States (6):
  source₊T(t) [defaults to 300]
  source₊p(t) [defaults to 101300.0]
  source₊qm(t) [defaults to 0]
  source₊ρ(t) [defaults to 1.2]
⋮
Parameters (0):

In [4]:
function TransitionPipe3(; name, n=10, f=0.011, D=1.0, L=1.0, T=300, p0=zeros(n), qm0=zeros(n))

    RT = 287.11 * T
    A0 = pi / 4 * D^2
    c10 = RT / A0
    c20 = c10 * f / 2 / D

    @named inlet = FlowPort()
    @named outlet = FlowPort()

    @parameters begin
        A = A0
        c1 = c10
        c2 = c20
        dx = L / n
        f = f
    end

    @variables (qm(t))[1:n] (p(t))[1:n+1]

    initialValue = Dict(qm[i] => qm0[i] for i = 1:n)
    merge!(initialValue, Dict(p[i] => p0[i] for i = 1:n))

    eqs_continous = [
        ∂(p[i]) ~ c1 * (qm[i-1] - qm[i]) / dx
        for i = 2:n
    ]

    eqs_momentum = [
        ∂(qm[i]) ~ (c1 * qm[i]^2 / (0.5 * (p[i+1] + p[i]))^2 - A) * (p[i+1] - p[i]) / dx + c1 * qm[i] / (0.5 * (p[i+1] + p[i])) * (qm[i-1] - qm[i+1]) / dx - c2 * qm[i] * abs(qm[i]) / (0.5 * (p[i+1] + p[i]))
        for i = 2:n-1
    ]

    bd = [
        p[1] ~ inlet.p
        p[n+1] ~ outlet.p
        qm[n] ~ -outlet.qm
        qm[1] ~ inlet.qm
        ∂(qm[1]) ~ (c1 * qm[1]^2 / (0.5 * (p[2] + p[1]))^2 - A) * (p[2] - p[1]) / dx + c1 * qm[1] / (0.5 * (p[2] + p[1])) * (3 * qm[1] - 4 * qm[2] + qm[3]) / dx - c2 * qm[1] * abs(qm[1]) / (0.5 * (p[2] + p[1]))
        ∂(qm[n]) ~ (c1 * qm[n]^2 / (0.5 * (p[n+1] + p[n]))^2 - A) * (p[n+1] - p[n]) / dx + c1 * qm[n] / (0.5 * (p[n+1] + p[n])) * (-3 * qm[n] + 4 * qm[n-1] - qm[n-2]) / dx - c2 * qm[n] * abs(qm[n]) / (0.5 * (p[n+1] + p[n]))
    ]
    compose(ODESystem([eqs_continous; eqs_momentum; bd], t; name=name, defaults=initialValue), inlet, outlet)
end

TransitionPipe3 (generic function with 1 method)

In [5]:
@named transPipe1 = TransitionPipe3(n=n, D=D, L=L, T=T, p0=p0, qm0=qm0, f=f)
@named transPipe2 = TransitionPipe3(n=n, D=D, L=L, T=T, p0=p0, qm0=qm0, f=f)

┌ Warning: inlet contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit /Users/jerell/.julia/packages/ModelingToolkit/hBVcX/src/systems/connectors.jl:40
┌ Warning: outlet contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit /Users/jerell/.julia/packages/ModelingToolkit/hBVcX/src/systems/connectors.jl:40


┌ Warning: inlet contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit /Users/jerell/.julia/packages/ModelingToolkit/hBVcX/src/systems/connectors.jl:40
┌ Warning: outlet contains 1 flow variables, yet 2 regular (non-flow, non-stream, non-input, non-output) variables. This could lead to imbalanced model that are difficult to debug. Consider marking some of the regular variables as input/output variables.
└ @ ModelingToolkit /Users/jerell/.julia/packages/ModelingToolkit/hBVcX/src/systems/connectors.jl:40


Model transPipe2 with 39 (41) equations
States (43):
  (p(t))[2] [defaults to 1.0e6]
  (p(t))[3] [defaults to 1.0e6]
  (p(t))[4] [defaults to 1.0e6]
  (p(t))[5] [defaults to 1.0e6]
⋮
Parameters (4):
  dx [defaults to 1.33333]
  c1 [defaults to 6.85425e5]
  c2 [defaults to 42839.0]
  A [defaults to 0.125664]

In [9]:
eqs = [
    connect(inletSource.source, transPipe1.inlet),
    connect(transPipe1.outlet, transPipe2.inlet),
    connect(transPipe2.outlet, outletSource.source),
    # connect(transPipe1.outlet, outletSource.source)

    # transPipe1.outlet.p ~ transPipe2.inlet.p,
    # transPipe1.outlet.qm ~ transPipe2.inlet.qm,
    # transPipe1.outlet.T ~ transPipe2.inlet.T,
    transPipe1.outlet.qv ~ transPipe2.inlet.qv,
]

4-element Vector{Equation}:
 connect(inletSource.source, transPipe1.inlet)
 connect(transPipe1.outlet, transPipe2.inlet)
 connect(transPipe2.outlet, outletSource.source)
 transPipe1₊outlet₊qv(t) ~ transPipe2₊inlet₊qv(t)

In [10]:
@named eq_model = ODESystem(eqs, t)
@named model = compose(
    eq_model,
    inletSource,
    outletSource,
    transPipe1,
    transPipe2
)

sys = structural_simplify(model)
prob = ODEProblem(sys, [], (0, 3))
sol = solve(prob, Rodas4())

retcode: InitialFailure
Interpolation: specialized 3rd order "free" stiffness-aware interpolation
t: 1-element Vector{Float64}:
 0.0
u: 1-element Vector{Vector{Float64}}:
 [1.0e6, 1.0e6, 1.0e6, 1.0e6, 1.0e6, 1.0e6, 1.0e6, 1.0e6, 1.0e6, 1.0e6  …  10.0, 10.0, 1.160995205089803, 4.643980820359212, 1.160995205089803, 857.2700728699768, 999999.998434483, 857.2700728699768, 4.0628802053328865, 4.643980820359212]